In [1]:
import cv2
import numpy as np
import os

from random import shuffle
from tqdm import tqdm

In [2]:
pwd

'C:\\Users\\Usha\\Documents\\GitHub\\computervision'

In [3]:
TRAIN_DIR='C:\\Users\\Usha\\Desktop\\images\\train'
TEST_DIR='C:\\Users\\Usha\\Desktop\\images\\test'

In [4]:
IMG_SIZE=50
LR=1e-3

In [5]:
MODEL_NAME = 'peoplecount-{}-{}.model'.format(LR, '2conv-basic')

In [6]:
def label_img(img):
    word_label = img.split('.')[0]
  
    if word_label == 'no': 
        return [1,0,0,0]
    elif word_label == 'one':
        return [0,1,0,0]
    elif word_label == 'two':
        return [0,0,1,0]
    elif word_label == 'three':
        return [0,0,0,1]
      

In [7]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir(TRAIN_DIR)):
        label = label_img(img)
        path = os.path.join(TRAIN_DIR,img)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

In [8]:
def process_test_data():
    testing_data = []
    for img in tqdm(os.listdir(TEST_DIR)):
        #label = label_img(img)
        path = os.path.join(TEST_DIR,img)
        img_num = img.split('.')[0]
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        #img = cv2.imread(path)
        img = cv2.resize(img, (IMG_SIZE,IMG_SIZE))
        testing_data.append([np.array(img)])
        
    shuffle(testing_data)
    np.save('test_data.npy', testing_data)
    return testing_data

In [9]:
train_data= create_train_data()

100%|███████████████████████████████████████| 385/385 [00:01<00:00, 210.60it/s]


In [10]:
import tensorflow as tf
import tflearn

C:\Users\Usha\Anaconda3\pyton\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [11]:
from tflearn.layers.conv import conv_2d, max_pool_2d
from tflearn.layers.core import input_data, dropout, fully_connected
from tflearn.layers.estimator import regression






convnet = input_data(shape=[None, IMG_SIZE, IMG_SIZE, 1], name='input')

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 128, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 64, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = conv_2d(convnet, 32, 5, activation='relu')
convnet = max_pool_2d(convnet, 5)

convnet = fully_connected(convnet, 1024, activation='relu')
convnet = dropout(convnet, 0.8)

convnet = fully_connected(convnet, 4, activation='softmax')
convnet = regression(convnet, optimizer='adam', learning_rate=LR, loss='categorical_crossentropy', name='targets')

model = tflearn.DNN(convnet, tensorboard_dir='log')



Instructions for updating:
Use tf.initializers.variance_scaling instead with distribution=uniform to get equivalent behavior.
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [12]:
if os.path.exists('{}.meta'.format(MODEL_NAME)):
    model.load(MODEL_NAME)
    print('model loaded!')


INFO:tensorflow:Restoring parameters from C:\Users\Usha\Documents\GitHub\computervision\peoplecount-0.001-2conv-basic.model
model loaded!


In [34]:
train = train_data[:-10]
test =train_data[100:200]
#train = train_data
#test = process_test_data()

In [35]:


X = np.array([i[0] for i in train]).reshape(-1,50,50,1)
Y = [i[1] for i in train]

test_x = np.array([i[0] for i in test]).reshape(-1,50,50,1)
test_y = [i[1] for i in test]



In [36]:
test_x

array([[[[254],
         [254],
         [134],
         ...,
         [143],
         [ 74],
         [140]],

        [[254],
         [161],
         [112],
         ...,
         [153],
         [ 72],
         [143]],

        [[254],
         [206],
         [226],
         ...,
         [160],
         [ 78],
         [146]],

        ...,

        [[ 94],
         [  6],
         [  6],
         ...,
         [ 39],
         [ 50],
         [ 25]],

        [[ 94],
         [ 30],
         [ 34],
         ...,
         [ 45],
         [ 41],
         [ 28]],

        [[201],
         [ 35],
         [ 41],
         ...,
         [ 41],
         [ 40],
         [ 21]]],


       [[[ 26],
         [ 27],
         [  9],
         ...,
         [  8],
         [  7],
         [  7]],

        [[ 23],
         [  8],
         [ 13],
         ...,
         [  8],
         [  9],
         [  9]],

        [[ 19],
         [ 11],
         [ 13],
         ...,
         [  8],
         [

In [37]:
model.fit({'input': X}, {'targets': Y}, n_epoch=50, validation_set=({'input': test_x}, {'targets': test_y}), 
    snapshot_step=20, show_metric=True, run_id=MODEL_NAME)
#model.fit({'input': X}, {'targets': Y}, n_epoch=10, snapshot_step=20, show_metric=True, run_id=MODEL_NAME)

Training Step: 899  | total loss: 0.00001 | time: 2.047s
| Adam | epoch: 100 | loss: 0.00001 - acc: 1.0000 -- iter: 320/375
Training Step: 900  | total loss: 0.00001 | time: 3.406s
| Adam | epoch: 100 | loss: 0.00001 - acc: 1.0000 | val_loss: 0.00000 - val_acc: 1.0000 -- iter: 375/375
--


In [38]:
model.save(MODEL_NAME)

INFO:tensorflow:C:\Users\Usha\Downloads\peoplecount-0.001-2conv-basic.model is not in all_model_checkpoint_paths. Manually adding it.


In [39]:
import matplotlib.pyplot as plt
test_data=process_test_data()


fig=plt.figure(figsize=(500,500))
img_list=[]
for num,data in enumerate(test_data):
    # cat: [1,0]
    # dog: [0,1]
    
    #img_num = data[1]
    img_data = data[0]
    #print(img_num)
    #y = fig.add_subplot(10,40,num+1)
    orig = img_data
    data = img_data.reshape(50,50,1)
    #model_out = model.predict([data])[0]
    model_out = model.predict([data])[0]
    print(model_out)
    if np.argmax(model_out) == 3: str_label='3'
    elif np.argmax(model_out) == 2: str_label='2'
    elif np.argmax(model_out) == 1: str_label='1'
    else : str_label='0'
    img_list.append(str_label)  
    #y.imshow(orig,cmap='gray')
    #plt.title(str_label)
    #y.axes.get_xaxis().set_visible(False)
    #y.axes.get_yaxis().set_visible(False)
#plt.show()

100%|███████████████████████████████████████| 566/566 [00:01<00:00, 351.27it/s]


[3.6452757e-10 4.6628705e-07 9.9999917e-01 3.2579288e-07]
[2.1948990e-10 5.4963566e-03 9.9450362e-01 3.6412338e-08]
[1.2381381e-09 5.7658644e-07 9.9999797e-01 1.4360444e-06]
[3.7688960e-11 5.1682659e-05 9.9994826e-01 8.0913258e-09]
[9.9027969e-13 9.9849409e-09 1.0000000e+00 2.5327247e-09]
[5.3352474e-09 4.1622539e-07 2.8334628e-06 9.9999678e-01]
[3.0102743e-12 5.9795710e-08 9.9999988e-01 4.4106887e-09]
[4.1623675e-12 6.3516609e-08 9.9999988e-01 6.5758936e-09]
[8.3202976e-06 4.7814005e-04 9.9864584e-01 8.6775306e-04]
[1.3426807e-11 6.4113628e-08 9.9999988e-01 9.8736388e-09]
[3.7352417e-12 9.3227209e-07 9.9999905e-01 1.2549680e-09]
[2.51467545e-11 3.34878337e-07 9.99999642e-01 1.47905945e-08]
[1.9272492e-10 4.3082420e-07 9.9999940e-01 7.3447183e-08]
[3.4783473e-10 3.3740594e-07 9.9999940e-01 2.7517268e-07]
[1.345887e-12 5.635552e-07 9.999994e-01 5.453672e-10]
[5.4950642e-14 2.6863700e-09 1.0000000e+00 1.1675191e-10]
[6.7331042e-14 5.1282253e-08 1.0000000e+00 5.3594931e-11]
[1.2254388e-10

[3.9729335e-12 2.4684889e-08 1.0000000e+00 3.6962504e-09]
[4.9367552e-11 1.0040729e-07 9.9999988e-01 4.8354078e-08]
[4.4612550e-11 1.0000000e+00 1.5655454e-08 1.7132763e-09]
[5.1007740e-12 3.4149814e-08 1.0000000e+00 1.6262215e-08]
[2.7119539e-11 1.0000000e+00 7.6738624e-09 1.0481341e-09]
[1.0640501e-11 1.0000000e+00 5.0698308e-09 4.8843929e-10]
[8.0426551e-14 1.6407080e-09 1.0000000e+00 2.0064594e-10]
[3.2394934e-11 4.9894306e-09 1.0275271e-07 9.9999988e-01]
[1.6476173e-12 5.9589461e-10 3.7485401e-09 1.0000000e+00]
[2.1780227e-11 3.8714791e-07 9.9999964e-01 6.5770465e-09]
[3.5638777e-11 9.9999976e-01 2.8020565e-07 1.8525141e-09]
[4.3095071e-14 7.6899759e-10 1.0000000e+00 1.8443906e-10]
[1.0245788e-13 2.9191170e-08 1.0000000e+00 8.7181193e-11]
[1.4425239e-11 3.6126803e-05 9.9996388e-01 3.5675005e-09]
[3.4381907e-11 6.8404447e-09 3.2549330e-08 1.0000000e+00]
[1.3199855e-10 1.5092041e-08 2.6211112e-07 9.9999976e-01]
[5.9282635e-12 1.5915100e-08 1.0000000e+00 1.5387947e-08]
[4.1742752e-11

[3.9095723e-11 1.8461211e-07 9.9999976e-01 4.4759290e-08]
[4.6272248e-12 4.6456634e-08 1.0000000e+00 9.2344727e-09]
[1.0101348e-12 1.4538343e-08 1.0000000e+00 2.3177136e-09]
[4.2957529e-13 4.3636103e-07 9.9999952e-01 2.1279672e-10]
[9.9999988e-01 8.6381846e-08 7.7361539e-10 1.5155317e-10]
[3.4469716e-10 1.7450905e-06 9.9999809e-01 6.0194161e-08]
[3.5140467e-13 1.0000000e+00 1.5455535e-09 3.1852576e-11]
[1.8208321e-07 3.4066616e-05 9.9993765e-01 2.8113871e-05]
[3.3103681e-10 3.5406277e-08 7.6255907e-07 9.9999917e-01]
[4.9104869e-12 4.4864773e-08 1.0000000e+00 4.4668877e-09]
[1.7056285e-11 6.5986008e-08 9.9999988e-01 1.2224847e-08]
[2.2162119e-11 1.0000000e+00 6.1348815e-09 8.6476060e-10]
[4.5285585e-13 4.6831334e-09 1.0000000e+00 1.2643924e-09]
[1.6843603e-11 1.3095209e-06 9.9999869e-01 4.1539621e-09]
[2.72996578e-12 1.13180185e-07 9.99999881e-01 2.68795364e-09]
[1.4221788e-11 1.4786798e-07 9.9999988e-01 2.6640931e-08]
[7.3729894e-12 1.1670711e-07 9.9999988e-01 7.6651139e-09]
[1.8198680

[1.0000000e+00 1.4251327e-08 7.1314354e-10 1.4802608e-10]
[9.4332576e-11 1.0714992e-08 3.6742588e-07 9.9999964e-01]
[1.1945788e-11 3.7388027e-08 1.0000000e+00 1.1654491e-08]
[1.1222663e-07 2.7791251e-05 9.9993050e-01 4.1554187e-05]
[8.2021624e-13 1.0492788e-08 1.0000000e+00 1.8874640e-09]
[1.9446540e-10 7.1949776e-06 9.9999285e-01 2.9938203e-08]
[3.4032520e-15 3.6425145e-09 1.0000000e+00 5.5607563e-12]
[6.2780510e-14 1.0000000e+00 6.9505384e-09 9.4933618e-12]
[6.2624878e-11 2.5424771e-07 9.9999976e-01 5.2134450e-08]
[1.1928108e-13 2.7858288e-08 1.0000000e+00 9.9547141e-11]
[1.1489972e-10 3.9516038e-07 9.9999964e-01 4.3219650e-08]
[2.3856436e-10 2.6160299e-08 3.4887188e-07 9.9999964e-01]
[4.9393523e-10 2.8220588e-06 9.9999702e-01 7.4050597e-08]
[1.5521018e-14 8.1942469e-10 1.0000000e+00 4.8179002e-11]
[5.1179356e-08 2.7891398e-05 9.9996555e-01 6.5737845e-06]
[2.0929684e-13 2.3764534e-07 9.9999976e-01 1.2225859e-10]
[1.5361399e-13 9.0932479e-09 1.0000000e+00 1.7830577e-10]
[3.5997498e-12

[7.8041124e-12 2.6306012e-08 1.0000000e+00 1.4734623e-08]
[6.9935370e-11 1.0000000e+00 2.7585965e-09 1.9739521e-09]
[5.7749914e-14 2.2893802e-08 1.0000000e+00 5.3525462e-11]
[1.6585265e-12 2.2711395e-08 1.0000000e+00 1.4598772e-09]


In [40]:
import os

In [41]:
os.chdir('C:\\Users\\Usha\\Downloads')

In [43]:
import cv2
testpath="C:\\Users\\Usha\\Desktop\\images\\test\\"
newpath="C:\\Users\\Usha\\Desktop\\images\\modified\\"



for num in enumerate(img_list):
    #read image
    name='frame'+str(num[0])
    name=name+'.jpg'
    modname='frame'+str(num[0])
    modname=modname+'.jpg'

    path1=os.path.join(testpath , name)
    path2=os.path.join(newpath , modname)
    image=cv2.imread(str(path1))
    cv2.putText(image,num[1], (125,125), cv2.FONT_HERSHEY_COMPLEX, 4, (0,0,255),cv2.LINE_AA)
    cv2.imwrite(str(path2),image)
    #print(num[0])
